# Coursera Data Science Capstone Project Week3

## This is a notebook to explore, segment, and cluster the neighborhoods in the city of Toronto.

### Data Preprocessing

In [1]:
#Import libraries
import pandas as pd
import numpy as np

First: screaping data from this website: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M with BeautifulSoup 

In [2]:
from bs4 import BeautifulSoup
import requests
import csv

#We assign the link of the website through which we are going to scrape the data
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#creating a BeautifulSoup (soup) object with the BeautifulSoup function
soup = BeautifulSoup(source, 'lxml')

In [3]:
#look into the "suop"
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":920980179,"wgRevisionId":920980179,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNames

In [4]:
#Taking the table what we need and look into it
postcode_table = soup.find('table', class_='wikitable sortable')
print(postcode_table.prettify())

<table class="wikitable sortable">
 <tbody>
  <tr>
   <th>
    Postcode
   </th>
   <th>
    Borough
   </th>
   <th>
    Neighbourhood
   </th>
  </tr>
  <tr>
   <td>
    M1A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M2A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M3A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Parkwoods" title="Parkwoods">
     Parkwoods
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M4A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Victoria_Village" title="Victoria Village">
     Victoria Village
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M5A
   </td>
   <td>
    <a href="/wiki/Downtown_Toronto" title="Downtown Toronto">
     Downtown Toronto
    </a>
   </td>
   <td>
    <a href="

In [5]:
#Take the row titles from the table and save in a list
rowtitles = []

for rowtitle in postcode_table.find_all('th'):
    rowtitles.append(rowtitle.text.rstrip())

print("The titles are:", rowtitles)

The titles are: ['Postcode', 'Borough', 'Neighbourhood']


In [6]:
#screaping the table values into a "content" list
content = []

for ths in postcode_table.find_all('tr'):
    content.append(ths.text.strip())

#Delete teh "\n"-s from the content list    
for i in range (0, len(content)):
    content[i] = content[i].replace('\n', ', ')

print(content)

['Postcode, Borough, Neighbourhood', 'M1A, Not assigned, Not assigned', 'M2A, Not assigned, Not assigned', 'M3A, North York, Parkwoods', 'M4A, North York, Victoria Village', 'M5A, Downtown Toronto, Harbourfront', 'M5A, Downtown Toronto, Regent Park', 'M6A, North York, Lawrence Heights', 'M6A, North York, Lawrence Manor', "M7A, Queen's Park, Not assigned", 'M8A, Not assigned, Not assigned', 'M9A, Etobicoke, Islington Avenue', 'M1B, Scarborough, Rouge', 'M1B, Scarborough, Malvern', 'M2B, Not assigned, Not assigned', 'M3B, North York, Don Mills North', 'M4B, East York, Woodbine Gardens', 'M4B, East York, Parkview Hill', 'M5B, Downtown Toronto, Ryerson', 'M5B, Downtown Toronto, Garden District', 'M6B, North York, Glencairn', 'M7B, Not assigned, Not assigned', 'M8B, Not assigned, Not assigned', 'M9B, Etobicoke, Cloverdale', 'M9B, Etobicoke, Islington', 'M9B, Etobicoke, Martin Grove', 'M9B, Etobicoke, Princess Gardens', 'M9B, Etobicoke, West Deane Park', 'M1C, Scarborough, Highland Creek', '

### It looks good, now create a dataframe from that list

In [7]:
#Create a dataframe from the first row
import pandas as pd
df = pd.DataFrame(content[1:], columns =['content'])
df.head()

,content
0,"M1A, Not assigned, Not assigned"
1,"M2A, Not assigned, Not assigned"
2,"M3A, North York, Parkwoods"
3,"M4A, North York, Victoria Village"
4,"M5A, Downtown Toronto, Harbourfront"


In [8]:
#Seprate the content column into three columns, and name it df_toronto
df_toronto = pd.DataFrame(df.content.str.split(',').tolist(),columns = rowtitles)

In [9]:
df_toronto.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [10]:
#Delete Borough with the value Not assigned
notAssigned = df_toronto['Borough'][0]

df_toronto = df_toronto[df_toronto["Borough"] != notAssigned]

df_toronto.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [11]:
#Change Neighbourhood = Not assigned -> Neighbourhood = Name of the Borough
for i in range(0, len(df_toronto)):
    if df_toronto.iloc[i]['Neighbourhood'] == notAssigned:
        df_toronto.iloc[i]['Neighbourhood'] = df_toronto.iloc[i]['Borough']

#Grouping Neighbourhoods by Postcode, values separating with ,
df_toronto = df_toronto.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join)

#Because the result will be a series, we have to convert into a dataframe again and add an index column to
df_toronto = df_toronto.to_frame().reset_index()

In [12]:
#The final dataset is
df_toronto.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village ..."
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [13]:
#Check a Not assigned value in the dataset

df_toronto[df_toronto['Postcode'] == "M7A"]

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [14]:
print("And the shape is:",df_toronto.shape[0], "rows and", df_toronto.shape[1], "columns.")

And the shape is: 103 rows and 3 columns.


### Taking the Toronto postal code coordinates

A csv data about the postal codes of Toronto is available on: http://cocl.us/Geospatial_data (Geospatial_Coordinates.csv)

In [15]:
df_latlon = pd.read_csv("Geospatial_Coordinates.csv")
df_latlon.rename(columns={'Postal Code':'Postcode'}, inplace=True)

In [16]:
df_latlon.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
#Add lat long values to the dataframe and name it df_sum
df_sum = pd.merge(df_toronto, df_latlon, on='Postcode')

In [18]:
df_sum.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Create a Folium map about Toronto

In [19]:
#Lat and Lon of Toronto
latitude = 	43.651070
longitude = -79.347015

In [20]:
#Import Folium library
import folium

In [22]:
#Create Toronto Map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_sum['Latitude'], df_sum['Longitude'], df_sum['Borough'], df_sum['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto